#  <p style="text-align: center;">Term paper for the Course:</p>
#  <p style="text-align: center;">REMOTE SENSING PRODUCTS FOR EARTH SYSTEM RESEARCH </p>



The following Jupyter notebook is the result of the work regarding the term paper for the course "Remote Sensing Products for  Earth System Research" at Leipzig University WS 23/24, led by Prof. Dr. Jian Peng.

The code here represents the full analysis done for this project, making the project easily reproducible. All necessary data is donwloaded within the notebook via google earth engine, except the climatic data from the dwd, as well as the shapefiles for the Nature Reserves Layer and the boundary of germany used for plotting. 

Regarding the **DWD data** temperature and preciptation data: 
 - It can be downloaded [here](https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/)
 - the data has to be downloaded and unzippedinto monthly folders named: '05_May', "06_Jun", "07_Jul", "08_Aug", "09_Sep"

Regarding the **shapefiles**:
  - The shapefiles for the Nature Reserves Layer can be downloaded [here](https://www.bfn.de/themen/natur-und-mensch/natur/naturschutzgebiete.html)
  - The shapefile for the boundary of germany can be downloaded [here](https://www.diva-gis.org/gdata)

**Folder structure**: 
- can be created using the code snippet found in the next cell 
- only a base path needs to be set 
- this notebook should then be moved to the process folder


**Note:** 

- some slight modifications to the code regarding file names or folder structure might still be necessary

``` python

import os

# Define base paths for data and output
base_path = ""  # path where to setup the project


# Setup paths
data_path = os.path.join(base_path, "data")
process_path = os.path.join(base_path, "process")
output_path = os.path.join(base_path, "output")
ndvi_data_path = os.path.join(data_path, "ndvi_seasonal")
temp_mean_data_path = os.path.join(data_path, "dwd", "temp_mean")
precip_data_path = os.path.join(data_path, "dwd", "precip")
shapes_data_path = os.path.join(data_path, "shapes")
corine_data_path = os.path.join(data_path, "corine")
output_path_mk_ndvi = os.path.join(output_path, "ndvi_mk")
output_path_climate_m   k = os.path.join(output_path, "climate_mk")


# List all paths that need to be checked and potentially created
paths_to_create = [
    ndvi_data_path,
    temp_mean_data_path,
    precip_data_path,
    shapes_data_path,
    corine_data_path,
    output_path_mk_ndvi,
    output_path_climate_mk,
    process_path
]

# Create the directories if they don't exist
for path in paths_to_create:
    os.makedirs(path, exist_ok=True)

```

*** 
#  <p style="text-align: center;">Drought stress trend Analysis for temperate Forests</p>


**Abstract**

Amidst two major vegetation events, namely global greening on a global scale and consecutive drought years in the recent past in Germany, this study investigates the impact of
climate change on German forests from 2000 to 2023. Remotely sensed data (MODISNDVI) of the summer month was utilized to assess forest health and dynamics. This study
identified significant trends in the NDVI, highlighting variations among forest types and
regions. Four regions (Black Forest, Harz Mountains, Rothaar Mountains, Thuringian
Slate-Mountains) were identified as especially impacted by a declining NDVI, with a
decrease notably pronounced in coniferous forests. The Mann-Kendall test, with time
offsets of zero, one, two, and three years, was used in these four regions to examine an
association between the NDVI and the climate parameters, temperature and precipitation,
however, no significant correlation was identified. This underscores the critical role of
local side factors for drought resistance and vulnerability, suggesting a need for fine-scale
forest management strategies to mitigate climate change impacts.


# Packages

In [ ]:
import os

import re
import tqdm

import numpy as np

import pandas as pd
import geopandas as gpd

import rioxarray
import xarray as xr
import rasterio
from rasterio.warp import Resampling
from rasterio.mask import mask


import pymannkendall as mk
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LinearSegmentedColormap, ListedColormap

# Reoccuring Functions

In [ ]:

########################################################
# 1. Function to genearate a raster from an array based on a reference raster
########################################################
# create raster from numpy
def create_trend_raster(template_file, np_2d_array, output_file, no_data_value=-9999):
    """
    Updated function to create a raster file from the trend test results or masks,
    ensuring the no_data_value is properly handled.

    Parameters:
    - template_file: Path to an NDVI TIFF file for metadata.
    - np_2d_array: 2D NumPy array with the data to write.
    - output_file: Path where the output raster will be saved.
    - no_data_value: The value to use for indicating no data in the output raster.
    """
    with rasterio.open(template_file) as src:
        meta = src.meta.copy()
        meta.update(dtype=rasterio.float32, count=1, nodata=no_data_value)

        with rasterio.open(output_file, 'w', **meta) as dst:
            # If input array is boolean, convert to float32 and use no_data_value for False
            if np.issubdtype(np_2d_array.dtype, np.bool_):
                np_2d_array = np.where(np_2d_array, 1.0, no_data_value).astype(rasterio.float32)
            else:
                np_2d_array = np_2d_array.astype(rasterio.float32)
                
            dst.write(np_2d_array, 1)



########################################################
# 2. Function to perform Mann-Kendall trend test
########################################################

def perform_mk_test_progress(data, no_data_value, test):
    """
    Perform the Mann-Kendall trend test on a 3D NumPy array.

    Parameters:
    - data: A 3D NumPy array with shape (time, height, width).
    - no_data_value: The value that indicates no data or missing data.

    Returns:
    - A tuple containing two 2D NumPy arrays:
      - Trend results (slope of the trend).
      - P-values for the trend test.
    """
    # Initialize arrays to store the test results and p-values
    trend_result = np.full((data.shape[1], data.shape[2]), no_data_value, dtype=np.float32)
    slope_result = np.full((data.shape[1], data.shape[2]), no_data_value, dtype=np.float32)
    trend_p_value = np.full((data.shape[1], data.shape[2]), no_data_value, dtype=np.float32)

    # Calculate total pixels for progress indicator
    total_pixels = data.shape[1] * data.shape[2]
    
    # Progress bar setup
    with tqdm.tqdm(total=total_pixels) as pbar:
        # Iterate over each pixel
        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                
                pbar.update(1)


                # Extract the time series for the current pixel
                pixel_series = data[:, i, j]

                # Skip the series if it contains the no_data_value
                if not np.any(pixel_series == no_data_value):
                    if not np.all(np.isnan(pixel_series)):  # Check if the pixel series is not all NaN


                        # Mask out no_data_values if necessary before test
                        clean_series = pixel_series[pixel_series != no_data_value] if np.any(pixel_series == no_data_value) else pixel_series

                        # Perform the test only if the cleaned series is not empty
                        if clean_series.size > 0:

                            if test == "seasonal":
                                        result = mk.seasonal_test(clean_series, period=5)
                            elif test == "auto":  # Changed to elif for proper flow
                                result = mk.correlated_seasonal_test(clean_series, period=5)
                            else:  # Fixed assignment typo
                                result = mk.original_test(clean_series)


                            slope_result[i, j] = result.slope
                            trend_result[i, j] = 1 if result.trend == "increasing" else -1 if result.trend == "decreasing" else 0
                            trend_p_value[i, j] = result.p

    return trend_result, trend_p_value, slope_result



# 0. Main Variables

In [ ]:
no_data_value = -9999
# Define CORINE land cover classes of interest
forest_classes = [311, 312, 313]


# as this script lays in the "project/process", we use relative pathing for the main project folder
base_path = (r"..")

# data path
data_path = os.path.join(base_path,"data")
ndvi_data_path = os.path.join(data_path, "ndvi_seasonal")

# output path
output_path = os.path.join(base_path, "output")

############  Input Paths DWD Data ############

temp_mean_data_path = os.path.join(data_path, "dwd", "temp_mean")
precip_data_path = os.path.join(data_path, "dwd", "precip")


############  Input Paths Vector data ############
shapes_data_path = os.path.join(data_path, "shapes")

germany_boundary_shp_path = os.path.join(shapes_data_path, "germany_v2.shp")
naturparke_shp_path = os.path.join(shapes_data_path, "Naturparke.shp")

############  Input Paths Corine Data ############
corine_data_path = os.path.join(data_path, "corine")

corine_combined_path = os.path.join(corine_data_path, "corine_combined.tif")






############  Output Paths Trend Analysis ############





# Define output file paths
output_path_mk_ndvi = os.path.join(output_path, "ndvi_mk")
output_path_trend_ndvi = os.path.join(output_path_mk_ndvi, "mk_trend_seasonal.tif")
output_path_slope_ndvi = os.path.join(output_path_mk_ndvi, "mk_slope_seasonal.tif")
output_path_sig_ndvi = os.path.join(output_path_mk_ndvi, "mk_sig_seasonal.tif")

# Output path for all mk tests of cliate data
output_path_climate_mk = os.path.join(output_path, "climate_mk")

############  template files (for meta data) for raster output creation ############

# get the first file in the ndvi_data_path
template_file_ndvi = os.path.join(ndvi_data_path, os.listdir(ndvi_data_path)[0])

# 1. Using GEE to preprocess and download NDVI and Corine Landcover Data

### Setup GEE

In [ ]:

# Load Packages and initalize GEE

import ee
import geemap
ee.Authenticate(force=False)
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com', project='ee-forest-health')


### 1.1 NDVI

Clip to germany boundary, Scale to -1 to 1, filter by year and calendar month (months 5-9)

In [ ]:

# Define a FeatureCollection for Germany using the FAO GAUL dataset, filtered for Germany's administrative level 0 (country level)
germany = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
germany_geometry = germany.geometry()
germany_mask = ee.Image().paint(germany, 1).mask()


modis_data_ndvi = (
    ee.ImageCollection('MODIS/061/MOD13A1')
    .select('NDVI')
    .filterDate("2000-01-01", "2024-01-01")
    .filter(ee.Filter.calendarRange(5, 9, 'month'))
)


################################## Masking with Forest ########################################
def apply_clip(image):
    return image.clip(germany_geometry)# .clip(forest_geometry)

# Function to divide the NDVI band values of an image by 10000
def apply_scale(image):
    ndvi_scaled = image.multiply(0.0001)
    return image.addBands(ndvi_scaled, overwrite=True)

ndvi_seasonal = modis_data_ndvi.map(apply_clip).map(apply_scale)
                                    


Download all files

In [ ]:
geemap.ee_export_image_collection(ndvi_seasonal, out_dir=ndvi_data_path, crs ="EPSG:25832", 
                                  scale=500, region = germany_geometry, file_per_band=True)

### 1.2 Corine Landcover

Define function to load corine landcover data for a specfic year, use forest classes only, clip to germany boundary, and output at 500 m resolution and with crs "EPSG:25832". Then locally it's clipped to the boundary of germany.

In [ ]:
# define function to get the landcover data
def download_preprocess_corine(landcover_collection, year, gdf):
    
    landcover_year = landcover_collection.filterDate(str(year-1)+'-01-01', str(year)+'-12-31').first()

    zones=ee.Image(0) \
   .where(landcover_year.eq(311), 311) \
   .where(landcover_year.eq(312), 312) \
   .where(landcover_year.eq(313), 313) 
    
    file_path_year = os.path.join(corine_data_path, str(year)+"_25832_500m.tif")

    geemap.ee_export_image(zones, 
                           filename = file_path_year, crs ="EPSG:25832", 
                           scale = 500, region = germany_geometry)
    

    print("Cropping to Germany shape")
    
    print(gdf.crs)
    

    # Open the TIFF file
    with rasterio.open(file_path_year) as src:
        # Crop the image

        print(src.crs)
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        
        # Copy the metadata
        out_meta = src.meta.copy()

   # Correctly update the metadata, especially dtype and nodata
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "crs": 'EPSG:25832',
        "dtype": src.meta['dtype'],  # Ensure dtype matches the original
        "nodata": src.meta['nodata']  # Preserve original nodata value
    })

    file_path_corine_cropped = file_path_year.replace(".tif","_cropped.tif")
    # Write the cropped image to a new file
    with rasterio.open(file_path_corine_cropped, 'w', **out_meta) as dest:
        dest.write(out_image)

    print("Remove non-cropped file")
    os.remove("file_name_year")
    
    return file_path_corine_cropped


Run function on corine Landcover data for 2000, 2006, 2012 and 2018

In [ ]:
# load landcover imagecollection
landcover_collection = ee.ImageCollection('COPERNICUS/CORINE/V20/100m')

# Load and the shapefile of the germany boundary
gdf_germany = gpd.read_file(germany_boundary_shp_path)
gdf_germany.crs = 'EPSG:25832'

corine_file_paths = []
years = [2000, 2006, 2012, 2018]
for year in years:
    file_path_corine_cropped = download_preprocess_corine(landcover_collection, year, gdf_germany)
    corine_file_paths.append(file_path_corine_cropped)

Now all corine landcover datasets will be compared to produce a final layer giving constant constant forest classes for each pixel. More precisely forest class 311 and 312 will only be given when the pixel showed the respective class in all four years. Otherwise the pixel will be given the class 313, mixed forest type.

In [ ]:

# Load raster data into arrays
arrays = []
for file in corine_file_paths:
    with rasterio.open(file) as src:
        profile = src.profile
        arrays.append(src.read(1))  # Assuming all rasters have one band

# Convert list of arrays into a 3D numpy array (bands, rows, cols)
stacked_arrays = np.stack(arrays)

# Using all() along axis=0 ensures comparison across all years for each pixel
corine_combined = np.where((stacked_arrays == stacked_arrays[0]).all(axis=0), stacked_arrays[0], 313)


with rasterio.open(corine_combined_path, 'w', **profile) as dst:
    dst.write(corine_combined.astype(profile['dtype']), 1)

# 2. Process Ndvi data and run seasonal Mann-Kendall test

Correct file names

In [ ]:
# add "_ndvi" at the beginning of each filename
for filename in os.listdir(ndvi_data_path):

    new_filename = "ndvi_" + filename
    os.rename(os.path.join(ndvi_data_path, filename), os.path.join(ndvi_data_path, new_filename))


Setup Ndvi montly means numpy array and mask with corine landcover forest class

In [ ]:
############  Load Ndvi Files ############

# List all TIFF files in the directory
ndvi_files = [f for f in os.listdir(ndvi_data_path) if f.endswith('.tif')]

# Initialize an empty list to hold your arrays
ndvi_arrays = []

# Loop through the files, read each one, and append the array to your list
for file in ndvi_files:
    with rasterio.open(os.path.join(ndvi_data_path, file)) as src:
        ndvi_arrays.append(src.read(1))  # read(1) reads the first band



############  Stack and Reshape ndvi data ############
        
# Stack the arrays into a single 3D numpy array
ndvi_stack = np.stack(ndvi_arrays, axis=0)

# Ensure the time dimension is divisible by 3
if ndvi_stack.shape[0] % 2 != 0:
    print("Time dimension is not divisible by 2. Please adjust the array.")
else:
    # Reshape the array to group every 3 timesteps
    reshaped_data = ndvi_stack.reshape(-1, 2, ndvi_stack.shape[1], ndvi_stack.shape[2])
    
    # Calculate the mean across the new axis (1) corresponding to the grouped timesteps
    ndvi_monthly_means = reshaped_data.mean(axis=1)
    



############  mask NDVI data ############

# Expand mask_2d to match array_3d's shape
corine_mask_3d = corine_combined[None, :, :]  # Adds a new axis, making it (1, x, y)

# Use broadcasting to apply the mask across all time steps
ndvi_masked_monthly_means = np.where(np.isin(corine_mask_3d, [311,312,313]),
                             ndvi_monthly_means,
                             no_data_value)  # Replace non-masked values with np.nan or other desired value

print(ndvi_masked_monthly_means.shape)

del ndvi_stack, corine_combined, reshaped_data, ndvi_arrays, stacked_arrays, arrays 

Run seasonal Man_kendall_test for each pixel and create raster output for significant trends (with direction) and slope

In [ ]:
# run MK
trend_result, trend_p_value, slope_result = perform_mk_test_progress(ndvi_masked_monthly_means, no_data_value, "seasonal")

# use trend_p_value to create mask
# Create a mask for significant trends (p < 0.025 or p>0.975)
significant_trend_mask = np.logical_or(trend_p_value < 0.025, trend_p_value > 0.975)


# safe results as tif
create_trend_raster(template_file_ndvi, trend_result, output_path_trend_ndvi)
create_trend_raster(template_file_ndvi, significant_trend_mask, output_path_sig_ndvi)
create_trend_raster(template_file_ndvi, slope_result, output_path_slope_ndvi)


# 3. Process Climate Data and run seasonal MK

For the next part DWD- montly gridded climate data was downloaded from the open climate data portal: https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/. The data was downloaded and unzipped manually. Montly folders per variable were created containing the files for all years for the respective variable and month. This led to the following code for preprocessing. 

Using Function created to reproject, mask and safe to tif for all files for one variable.

In [ ]:

########################################################
# Function to reproject and mask a raster used for processing DWD Data
########################################################

def reproject_and_mask_rasters_dwd(data_folder, file_prefix, target_crs, corine_path, mask_crs_values=[311, 312, 313], source_crs = "EPSG:31467"):
    """
    Reproject rasters to a target CRS and mask based on specified values.
    
    Parameters:
    - base_dir: Directory containing the raster files.
    - file_prefix: Prefix to identify specific raster files.
    - target_crs: CRS to reproject the rasters to, e.g., 'EPSG:25832'.
    - mask_crs_values: List of values in the mask raster to use for masking.
    """
    month_folders = ['05_May', "06_Jun", "07_Jul", "08_Aug", "09_Sep"]
    reprojected_file_paths = []  # Stores paths to the reprojected and masked TIFF files
    
    for month in month_folders:
        month_number = month[:2]
        month_dir = os.path.join(data_folder, month)
        if os.path.isdir(month_dir):
            for filename in os.listdir(month_dir):
                if filename.startswith(f"{file_prefix}_{month_number}") and filename.endswith(".asc"):
                    file_path = os.path.join(month_dir, filename)
                    output_tiff_path = os.path.splitext(file_path)[0] + '_reprojected_masked.tif'
                    
                    # Load the raster data
                    raster_data = rioxarray.open_rasterio(file_path, masked=True)
                    raster_data.rio.set_crs(source_crs, inplace=True)

                    # Reproject the raster to the target CRS
                    reprojected_data = raster_data.rio.reproject(target_crs)

                    # Assuming 'mask_raster_path' is the path to your mask raster
                    # Load the mask raster and reproject to match the reprojected data
                    mask_raster = rioxarray.open_rasterio(corine_path, masked=True)
                    mask_reprojected = mask_raster.rio.reproject_match(reprojected_data, resampling=Resampling.nearest)

                    # Apply mask based on specified values
                    mask_condition = np.isin(mask_reprojected, mask_crs_values)
                    masked_data = reprojected_data.where(mask_condition, np.nan)
                        
                    # Save the reprojected and masked raster
                    masked_data.rio.to_raster(output_tiff_path)
                    reprojected_file_paths.append(output_tiff_path)
                    
    return reprojected_file_paths





In [ ]:

# preprocessing for Temperature Data
temp_tifs_path = reproject_and_mask_rasters_dwd(data_folder =temp_mean_data_path,
                                            file_prefix = "TAMM", target_crs = "EPSG:25832", corine_path = corine_combined_path)

# preprocessing for Precipitation Data
precip_tifs_path = reproject_and_mask_rasters_dwd(data_folder =precip_data_path,
                                            file_prefix = "RSMS", target_crs = "EPSG:25832", corine_path = corine_combined_path)

Functions for DWD data processing and seasonal Mann-Kendall test

In [ ]:
############  Function to create output paths ############
def create_output_paths(variable, data_path, output_path):
    variable_path = os.path.join(data_path, variable)

    output_path_trend= os.path.join(output_path, variable, "mk_trend_" + variable + ".tif")
    output_path_slope = os.path.join(output_path, variable,"mk_slope_" + variable + ".tif")
    output_path_sig = os.path.join(output_path, variable,"mk_sig_" + variable + ".tif")
    
    #print output path
    return(output_path_trend, output_path_slope, output_path_sig, variable_path)

############  Function to extract date from filename ############

# option date_time_reverse is used to extract the date from the filename in the format "YYYY_MM_DD" instead of "MM_YYYY" which is used for the NDVI files
def extract_date(filename, for_ndvi=False):
    # Regular expression to find the date in the filename
    
    if for_ndvi:
        match = re.search(r'(\d{4})_(\d{2})_(\d{2})', filename)
        if match:
            # Return a tuple (Year, Month) for correct chronological sorting
            return (match.group(1), match.group(2), match.group(3))
        else:
            return ('')

    else:
        match = re.search(r'(\d{2})_(\d{4})', filename)

        if match:
            # Return a tuple (Year, Month) for correct chronological sorting
            return (match.group(2), match.group(1))
        else:
            return ('')


############  Function to create data_stack ############

def create_data_stack(variable_path, no_data_value = -9999): 
    # List all TIFF files in the directory
    data_paths_unordered = [f for f in os.listdir(variable_path) if f.endswith('.tif')]


    # Sorting the filenames using the extracted data
    data_files = sorted(data_paths_unordered, key=lambda x: extract_date(x))

    print(data_files[0])
    print(data_files[1])

    template_file = os.path.join(variable_path, data_files[0])

    # Initialize an empty list to hold your arrays
    clim_arrays = []

    # Loop through the files, read each one, and append the array to your list
    for file in data_files:
        with rasterio.open(os.path.join(variable_path, file)) as src:
            data_array = src.read(1)
            data_array = np.nan_to_num(data_array, nan=no_data_value)
            clim_arrays.append(data_array)

    # Stack the arrays into a single 3D numpy array
    clim_stack = np.stack(clim_arrays, axis=0)

    print(clim_stack.shape)
    return(clim_stack, template_file)
  

############  Main Function for processing and analysis for DWD climate Data ############

def main(variable_number, data_path, output_path, no_data_value = -9999, variables = ["temp_mean","precip"]):
    
    # variable
    variable = variables[variable_number]
    print(variable)
    
    
    
    # create output paths
    print("Create output paths")
    output_path_trend, output_path_slope, output_path_sig, variable_path = create_output_paths(variable, data_path, output_path)
    
    # print output paths
    print("Output paths: ", output_path_trend, output_path_slope, output_path_sig)

    # create data stack
    print("Create data stack")
    data_stack, template_file = create_data_stack(variable_path, no_data_value = no_data_value)
    
    # print data stack
    print("Data stack: ", data_stack.shape)
    print("Template file: ", template_file)

    # perform mk test
    print("Perform MK test")
    trend_result, trend_p_value, slope_result = perform_mk_test_progress(data_stack, no_data_value, test = "seasonal")
    significant_trend_mask = np.logical_or(trend_p_value < 0.025, trend_p_value > 0.975)


    # create output raster
    print("Create output raster")
    create_trend_raster(template_file, trend_result, output_path_trend)
    create_trend_raster(template_file, slope_result, output_path_slope)
    create_trend_raster(template_file, significant_trend_mask, output_path_sig)

    print("done")
    return output_path_trend, output_path_slope, output_path_sig

Defining variables and Executing function for all three climate variables. 

# For this step 

In [ ]:
# variable folders
dwd_variables = ["temp_mean","precip",]

# For temperature
output_path_trend_temp, output_path_slope_temp, output_path_sig_temp = main(0, data_path, output_path_climate_mk, no_data_value = -9999, variables = dwd_variables)

# For precipitation
output_path_trend_precip, output_path_slope_precip, output_path_sig_precip = main(1, data_path, output_path_climate_mk, no_data_value = -9999, variables = dwd_variables)


# 4. Correlation Analysis 

For the correlation analysis we will look at 6 german forest areas. For each forest area a mean annual NDVI time series will be correlated with mean annual time series of our three climatic variables. 

### 4.0 Define Functions

We will now start working with xarray datasets

In [ ]:
############  Function to create xarray dataarrayfrom files and reshape to annual means ############

def create_data_stack_xarray(variable, no_data_value=-9999, for_ndvi=False, show_months = False, ndvi_slope_path = output_path_slope_ndvi):
    variable_path = os.path.join(data_path, variable)

    # List all TIFF files in the directory
    data_paths_unordered = [f for f in os.listdir(variable_path) if f.endswith('.tif')]

    # ndvi slope to use only negative values
    if for_ndvi:
        ndvi_slope = rioxarray.open_rasterio(ndvi_slope_path)

    # Sorting the filenames using the extracted data
    data_files = sorted(data_paths_unordered, key=lambda x: extract_date(x, for_ndvi))

    # Initialize an empty list to hold your DataArrays
    clim_dataarrays = []
    
    # Initialize a list to hold the time coordinates
    time_coords = []

    # Loop through the files, open each one as a DataArray, and append to the list
    for file in data_files:
        da = rioxarray.open_rasterio(os.path.join(variable_path, file))
        # Replace no data values (-9999) with NaN directly
        da = da.where(da != no_data_value, np.nan)
        clim_dataarrays.append(da)
        
        # Extract date from filename and convert to datetime
        date_info = extract_date(file, for_ndvi)
        if for_ndvi:
            year, month, day = date_info
        else:
            year, month = date_info
            day = 1
        time_coords.append(pd.Timestamp(year=int(year), month=int(month), day=int(day)))

    # Concatenate all DataArrays along a new dimension called 'time'
    clim_stack_xarray = xr.concat(clim_dataarrays, dim='time')
    
    # Set the time coordinate
    clim_stack_xarray = clim_stack_xarray.assign_coords(time=('time', time_coords))

    if for_ndvi:
        corine_combined = rioxarray.open_rasterio(corine_combined_path)
        land_cover_mask = corine_combined.isin([311, 312, 313])
        land_cover_mask_3d = land_cover_mask.broadcast_like(clim_stack_xarray.isel(time=0))
        # Apply land cover mask and replace values outside the mask with np.nan
        clim_stack_xarray = xr.where(land_cover_mask_3d, clim_stack_xarray, np.nan)
        clim_stack_xarray = clim_stack_xarray.where(ndvi_slope < 0, np.nan)

    # Group by year and list months for each year
    grouped_by_year = clim_stack_xarray.groupby('time.year')

    if show_months:
        for year, group in grouped_by_year:
            months = group.time.dt.month.values
            print(f"Year: {year}, Months: {months}")


    # Calculate annual means, ensuring no_data_value is correctly handled as np.nan
    clim_stack_xarray_annual_means = clim_stack_xarray.resample(time='AS').mean(skipna=True)
    clim_stack_xarray_annual_means.name = variable

    return clim_stack_xarray_annual_means


### 4.1 Create Xarrays datasets and reshape to annual means

In [ ]:
temp_mean_xr = create_data_stack_xarray(variable = "temp_mean", no_data_value = -9999)
precip_xr = create_data_stack_xarray(variable = "precip", no_data_value = -9999)
ndvi_xr = create_data_stack_xarray(variable = "ndvi_seasonal", no_data_value = -9999, for_ndvi= True, show_months=False)

### 4.2 Load and process forest area shapefiles

In [ ]:
gebiete = [["Sauerland-Rothaargebirge","Arnsberger Wald", "Bergisches Land","Lahn-Dill-Bergland"],
           ["Harz/Sachsen-Anhalt", "Südharz","Harz/Niedersachsen","Harz Sachsen-Anhalt (Mansfelder Land)"],
           ["Südschwarzwald"],
           ["Thüringer Schiefergebirge/Obere Saale","Thüringer Wald","Frankenwald"]]


# load naturparke with gpd
gdf_naturparke = gpd.read_file(naturparke_shp_path)

# transform to epsg 25832
gdf_naturparke = gdf_naturparke.to_crs("EPSG:25832")

In [ ]:
# List to store the merged polygons
merged_polygons = []

for gemeinde_names in gebiete:
    # Subset the GeoDataFrame for each name in the sublist
    subset_gdf = gdf_naturparke[gdf_naturparke['NAME'].isin(gemeinde_names)]
    
    # Merge the polygons found in the subset if there's any, otherwise skip
    if not subset_gdf.empty:
        merged_polygon = subset_gdf.unary_union
        # Store the merged polygon with a reference to the gemeinde names for identification
        merged_polygons.append((gemeinde_names, merged_polygon))

# To convert the list of merged polygons back into a GeoDataFrame
# First, create a list of dictionaries with the appropriate structure
data = [{'NAME': names, 'geometry': polygon} for names, polygon in merged_polygons]
# Then, convert this list into a GeoDataFrame
merged_gdf_naturparke = gpd.GeoDataFrame(data, crs=gdf_naturparke.crs)


In [ ]:
merged_gdf_naturparke.NAME = ["Rothaargebirge","Harz","Schwarzwald_sued_mitte","Thüringer_Schiefergebirge"]

### 4.3 Calculate means over polygons

In [ ]:
def polygon_mean_over_time(xarrays, gdf, corine_combined_path):
    """
    Calculates mean values for each xarray.DataArray within polygons defined in a GeoDataFrame for each timestep,
    with keys in the output dictionary being the 'NAME' of the polygons.

    Parameters:
    - xarrays: List of xarray.DataArray objects. Each DataArray should have a 'time' dimension.
    - gdf: A GeoDataFrame containing polygons, each with a 'NAME' attribute.

    Returns:
    - A dictionary with keys as polygon names from the GeoDataFrame 'NAME' column and values as Pandas DataFrames.
      Each DataFrame contains mean values for each timestep as rows and each DataArray as columns.
    """
    results = {}
    for _, polygon in gdf.iterrows():
        polygon_name = polygon['NAME']
        # Initialize a DataFrame to hold results for the current polygon by its name
        polygon_means = {da.name: [] for da in xarrays}  # Assuming each DataArray has a 'name' attribute
        

        for da in xarrays:

       
            da.rio.write_crs("EPSG:25832", inplace=True)  # Example with EPSG:4326, adjust CRS as neede


            # Clip the DataArray by the current polygon
            clipped = da.rio.clip([polygon.geometry], crs=gdf.crs, drop=True, all_touched=True)
            # Calculate the mean for each time step and store it
            for time in clipped.time:
                mean_val = clipped.sel(time=time).mean(skipna=True).values.item()
                polygon_means[da.name].append(mean_val)
   
        # Convert the lists of means into a DataFrame with a time index
        index = pd.to_datetime([str(time.values)[:10] for time in clipped.time])  # Adjust string slicing as necessary
        results[polygon_name] = pd.DataFrame(polygon_means, index=index)
    
    return results

In [ ]:
xarrays = [temp_mean_xr, precip_xr, ndvi_xr]

In [ ]:
climate_ndvi_annual_series= polygon_mean_over_time(xarrays, merged_gdf_naturparke, corine_combined_path)


In [ ]:
climate_ndvi_annual_series

### 4.4 Analyze correlations between ndvi and climate variables

In [ ]:

def calculate_correlations_and_significance_table_with_lag(results_dict, lag=0):
    """
    Calculates the correlation and its significance of 'ndvi_seasonal' with the other three DataArrays for each polygon,
    including the option to add a lag.

    Parameters:
    - results_dict: The output from the `polygon_mean_over_time` function.
    - lag: The lag to apply to 'ndvi_seasonal' before correlation calculation.

    Returns:
    - A Pandas DataFrame with polygon names as columns, rows for correlations and p-values for each DataArray comparison, considering the specified lag.
    """
    
    # Initialize lists to store data
    data = {"Metric": []}
    polygons = list(results_dict.keys())
    for polygon_name in polygons:
        data[polygon_name] = []
    
    for column in next(iter(results_dict.values())).columns:
        
        if column != 'ndvi_seasonal':  # Skip 'ndvi_seasonal' itself
            data["Metric"].extend([f"Correlation with {column} (lag={lag})", f"P-value with {column} (lag={lag})"])
            for polygon_name, df in results_dict.items():
                # Apply lag
                ndvi_seasonal_lagged = df['ndvi_seasonal'].shift(lag)
                # Calculate correlation and p-value with lag
                corr, p_value = pearsonr(ndvi_seasonal_lagged.dropna(), df[column][lag:].dropna())
                data[polygon_name].extend([corr, p_value])
                
    # Convert the dictionary to a DataFrame
    results_df = pd.DataFrame(data).set_index("Metric")
    return results_df


In [ ]:
climate_ndvi_annual_series

In [ ]:
# Assuming `results` is your output from `polygon_mean_over_time`
correlations_0 = calculate_correlations_and_significance_table_with_lag(climate_ndvi_annual_series, lag =0)

correlations_1 = calculate_correlations_and_significance_table_with_lag(climate_ndvi_annual_series, lag =1)

correlations_2 = calculate_correlations_and_significance_table_with_lag(climate_ndvi_annual_series, lag =2)

correlations_3 = calculate_correlations_and_significance_table_with_lag(climate_ndvi_annual_series, lag =3)



# Now `correlations` contains the correlation of 'ndvi_seasonal' with each of the other arrays for each polygon

# give mean values over rows

print("Lag 0:",correlations_1.mean(axis=1),"\n")
print("Lag 1:",correlations_1.mean(axis=1),"\n")
print("Lag 2:",correlations_2.mean(axis=1),"\n")
print("Lag 3:",correlations_3.mean(axis=1),"\n")

# 5. Layers with trend direction and signifcance

For ndvi, temperature and precipitation produce a layer with information on trend direction and significance.

In [ ]:

def slope_trend_to_sig_trend(output_path_slope, output_path_trend, no_data_value=-9999):
    # Load the slope and trend raster datasets with masking enabled for no_data values
    slope = rioxarray.open_rasterio(output_path_slope, masked=True)
    trend = rioxarray.open_rasterio(output_path_trend, masked=True)

    # Create a new xarray DataArray for significant trends initialized with the no_data_value
  # Initialize the output dataset with the same shape and coordinates as the input, filled with no_data_value
    sig_trends = xr.full_like(slope, no_data_value)

    # Apply conditions to determine significant trends
    # Positive slope and trend == 1
    sig_trends = xr.where((slope > 0) & (trend == 1), 2, sig_trends)
    # Positive slope and trend == 0
    sig_trends = xr.where((slope > 0) & (trend == 0), 1, sig_trends)
    # Negative slope and trend == -1
    sig_trends = xr.where((slope < 0) & (trend == -1), -2, sig_trends)
    # Negative slope and trend == 0
    sig_trends = xr.where((slope < 0) & (trend == 0), -1, sig_trends)

    sig_trends.rio.write_crs(trend.rio.crs, inplace=True)
    sig_trends.rio.set_nodata(no_data_value, inplace=True)

    # Save the significant trends raster to a new file
    # The output path is derived from the slope file path to avoid overwriting original data
    output_path_sig_trends = output_path_slope.replace("slope", "combined_trend_slope")
    sig_trends.rio.to_raster(output_path_sig_trends)

    # Print the path of the saved significant trends raster for verification
    return output_path_sig_trends

In [ ]:
output_path_sig_trends_ndvi = slope_trend_to_sig_trend(output_path_slope_ndvi, output_path_trend_ndvi, no_data_value = -9999)

output_path_sig_trends_temp = slope_trend_to_sig_trend(output_path_slope_temp, output_path_trend_temp, no_data_value = -9999)

output_path_sig_trends_precip = slope_trend_to_sig_trend(output_path_slope_precip, output_path_trend_precip, no_data_value = -9999)

# 6. Analyzing ndvi Trend distribution over forest classes per forest area

In [ ]:

corine_combined = rioxarray.open_rasterio(corine_combined_path, masked=True)
trend_ndvi = rioxarray.open_rasterio(output_path_trend_ndvi, masked=True)




def calculate_trend_statistics(corine_combined, sig_trend_path, polygon = None, germany=False, forest_classes=[311, 312, 313]):
    sig_trend = rioxarray.open_rasterio(sig_trend_path, masked=True)

    if germany:
        corine_combined_crop = corine_combined
        sig_trend_crop = sig_trend
    else:   
        print(polygon["NAME"])
        corine_combined_crop = corine_combined.rio.clip([polygon.geometry], "EPSG:25832", drop=True, all_touched=True)
        sig_trend_crop = sig_trend.rio.clip([polygon.geometry], "EPSG:25832", drop=True, all_touched=True)

    print(np.unique(sig_trend_crop))

    forest_class_list = []
    positive_trend_percentage_list = []
    negative_trend_percentage_list = []
    sig_positive_trend_percentage_list = []
    sig_negative_trend_percentage_list = []

    for land_cover_class in forest_classes:
        class_mask = (corine_combined_crop == land_cover_class)

        positive_trend_count = np.sum((sig_trend_crop == 1) & class_mask)
        sig_positive_trend_count = np.sum((sig_trend_crop == 2) & class_mask)
        
        negative_trend_count = np.sum((sig_trend_crop == -1) & class_mask)
        sig_negative_trend_count = np.sum((sig_trend_crop == -2) & class_mask)

        total_pixels = np.sum(class_mask)

        # Calculate percentages and directly convert to scalar values for appending
        positive_trend_percentage = (positive_trend_count / total_pixels).item() if total_pixels else np.nan
        sig_positive_trend_percentage = (sig_positive_trend_count / total_pixels).item() if total_pixels else np.nan

        negative_trend_percentage = (negative_trend_count / total_pixels).item() if total_pixels else np.nan
        sig_negative_trend_percentage = (sig_negative_trend_count / total_pixels).item() if total_pixels else np.nan

        forest_class_list.append(land_cover_class)
        positive_trend_percentage_list.append(np.round(positive_trend_percentage * 100, 1))
        negative_trend_percentage_list.append(np.round(negative_trend_percentage * 100, 1))

        sig_positive_trend_percentage_list.append(np.round(sig_positive_trend_percentage * 100, 1))
        sig_negative_trend_percentage_list.append(np.round(sig_negative_trend_percentage * 100, 1))

    # Create the DataFrame with correct column assignments
    results_df = pd.DataFrame({
        'Forest Class': forest_class_list,
        'Positive Trend Percentage': positive_trend_percentage_list,
        'Negative Trend Percentage': negative_trend_percentage_list,
        'Significant Positive Trend Percentage': sig_positive_trend_percentage_list,
        'Significant Negative Trend Percentage': sig_negative_trend_percentage_list
    })

    return results_df



In [ ]:
# Initialize an empty DataFrame for the aggregated results
aggregated_results = pd.DataFrame()


############  Calculate trend statistics for germany ############

germany_results = calculate_trend_statistics(corine_combined, output_path_sig_trends_ndvi, germany=True)
germany_results['Area'] = "Germany"
aggregated_results = pd.concat([aggregated_results, germany_results], axis=0)



############  Calculate Trend Statistics for forest areas ############

# Iterate over each polygon
for index, polygon in merged_gdf_naturparke.iterrows():
    # Calculate trend statistics for the current polygon
    current_results = calculate_trend_statistics(corine_combined, output_path_sig_trends_ndvi, polygon)
    
    # Insert polygon name as a column for identification
    current_results['Area'] = polygon["NAME"]
    
    # Append the results to the aggregated DataFrame
    aggregated_results = pd.concat([aggregated_results, current_results], axis=0)



# Set a new index based on 'Polygon Name' and 'Forest Class'
aggregated_results.set_index(['Area', 'Forest Class'], inplace=True)

aggregated_results

# 7. Plotting Results

### Variables

In [ ]:
############  Preparing Data ############


gdf_germany = gpd.read_file(germany_boundary_shp_path)
gdf_germany.crs = 'EPSG:25832'
polygon_gdf = gpd.GeoDataFrame(merged_gdf_naturparke, geometry='geometry', crs=merged_gdf_naturparke.crs)

corine_combined_clean = corine_combined.where(corine_combined.isin([311, 312, 313]), np.nan)

ndvi_trend_slope_combined = rioxarray.open_rasterio(output_path_sig_trends_ndvi)
temp_trend_slope_combined = rioxarray.open_rasterio(output_path_sig_trends_temp)
precip_trend_slope_combined = rioxarray.open_rasterio(output_path_sig_trends_precip)

ndvi_trend_slope_combined = ndvi_trend_slope_combined.where(ndvi_trend_slope_combined != no_data_value)
temp_trend_slope_combined = temp_trend_slope_combined.where(temp_trend_slope_combined != no_data_value)
precip_trend_slope_combined = precip_trend_slope_combined.where(precip_trend_slope_combined != no_data_value)


ndvi_slope = rioxarray.open_rasterio(output_path_slope_ndvi)
temp_slope = rioxarray.open_rasterio(output_path_slope_temp)
precip_slope = rioxarray.open_rasterio(output_path_slope_precip)

ndvi_slope = ndvi_slope.where(ndvi_slope != no_data_value)
temp_slope = temp_slope.where(temp_slope != no_data_value)
precip_slope = precip_slope.where(precip_slope != no_data_value)


############  Preparing colors and legend entries etc. ############

# creating a arrays containing the relevant legend labels
legend_5_labels = ["sig. negative", "negative (-)", "neutral", "positive (+)", "sig. positive"]
legend_slope = ["neg (-)","neutral"," pos (+)"]

# defining all the needed colorlists
color_list_temp = ["#bd0026", "#f03b20", "#fd8d3c", "#fecc5c", "#ffffb2"]
color_list_plasma = ["#0d0887", "#7e03a8", "#cc4778", "#f89540", "#f0f921"]
color_map_plasma =  ListedColormap(color_list_plasma)
color_map_plasmaSeg = LinearSegmentedColormap.from_list("color_list_temp", color_list_temp)

color_list_ndvi = ["#d01c8b", "#f1b6da", "#f7f7f7", "#b8e186", "#4dac26"]
color_map_ndvi_slope_trend = ListedColormap(color_list_ndvi)
color_map_ndvi = LinearSegmentedColormap.from_list("color_list_ndvi", color_list_ndvi)

color_list_precip = ["#a6611a", "#dfc27d", "#f5f5f5", "#80cdc1", "#018571"]
color_map_precip = LinearSegmentedColormap.from_list("color_list_precip", color_list_precip)
color_map_precip_slope_trend =  ListedColormap(color_list_precip)


### Funcion

In [ ]:
######################### plotting fuction WITH flexible legend labels ###########################################


# setting up the function, including geodata, boundaries and more for the map
def plot_data_flex(dataset, cmap, vmin, vmax, plot_polygons=False, map_title='', legend_labels=None):
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot Germany with gray fill and black boundary
    gdf_germany.plot(ax=ax, color='lightgray', edgecolor='black', alpha=1, linewidth=1.5)

    # Plot rasterio dataset
    dataset.plot(ax=ax, alpha=1, vmin=vmin, vmax=vmax, cmap=cmap, add_colorbar=False)

    if plot_polygons:
        # Plot polygon boundaries
        polygon_gdf.boundary.plot(ax=ax, color='black', linewidth=1, linestyle='dashdot')

    # Remove x and y axis ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Set the map title if specified
    if map_title:
        ax.set_title(map_title, fontweight='bold') # Title in bold
        # larger title
        ax.title.set_fontsize(13)

    # Remove the border around the map
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    # Create a colorbar
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=Normalize(vmin=vmin, vmax=vmax), cmap=cmap),
                        ax=ax, fraction=0.02, pad=0.04)
    
    # Adjust the colorbar to work with both groups of 3 and 5 legend labels
    if legend_labels:
        # Calculate evenly spaced ticks between vmin and vmax based on the number of labels
        ticks = np.linspace(vmin, vmax, len(legend_labels))
        cbar.set_ticks(ticks)
        cbar.ax.set_yticklabels(legend_labels)  # Set custom legend labels
    else:
        # Default case for 3 labels if legend_labels is not specified
        cbar.set_ticks([vmin, (vmin+vmax)/2, vmax])
        cbar.ax.set_yticklabels([str(vmin), str((vmin+vmax)/2), str(vmax)])  # Default numeric labels
    
    # Adding a north arrow
    ax.annotate('N', xy=(1.1,0.99), xytext=(1.1, 0.9),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                xycoords='axes fraction', textcoords='axes fraction',
                fontsize=20, ha='center')

    plt.show()

### Creating Plots

In [ ]:
# Plotting CLC forest classes and our designated AOIs
plot_data_flex(corine_combined_clean, ListedColormap(['#66a61e', '#1b9e77', '#7570b3']), 311, 313, 
    plot_polygons = True,
    map_title=' ', #'Combined CLC Forest Classes and AOIs'
    legend_labels=['Decidious Forest', 'Coniferous Forest', 'Mixed Forest'])



#################SLOPE###################
#slope of ndvi
plot_data_flex(dataset = ndvi_slope, cmap= color_map_ndvi, vmin= -0.002, vmax = 0.002, 
    plot_polygons=True, 
    map_title=' ',#'Slope of NDVI'
    legend_labels=legend_slope)

#slope of temperature
plot_data_flex(dataset = temp_slope, cmap = 'plasma', vmin= -1, vmax = 1, 
    plot_polygons=True,
    map_title=' ',#'Slope of temperature'
    legend_labels=legend_slope)

#slope of precipitation
plot_data_flex(dataset = precip_slope, cmap= color_map_precip, vmin= -1, vmax = 1,
    plot_polygons=True,
    map_title=' ' ,#'Slope of precipitation'
    legend_labels=legend_slope)


##########TREND DIR AND SIGN###############
# for ndvi
plot_data_flex(dataset = ndvi_trend_slope_combined, cmap= color_map_ndvi_slope_trend, 
    vmin= -2, vmax = 2, plot_polygons=True, map_title=' ' #'Combined slope and trend for NDVI'
    , legend_labels=legend_5_labels)

#for temperature
plot_data_flex(dataset = temp_trend_slope_combined, cmap= color_map_plasma 
    , vmin= -1, vmax = 2
    , plot_polygons=True, map_title=' '#'Combined slope and trend for temperature'
    , legend_labels=legend_5_labels)

# for precipitation
plot_data_flex(dataset = precip_trend_slope_combined, cmap= color_map_precip_slope_trend, 
    vmin= -1.7, vmax = 1.7, plot_polygons=True, map_title= ' ' #'Combined slope and trend for precipitation'
    , legend_labels=legend_5_labels)